In [1]:
import pandas as pd
import numpy as np
import os
import re
import time

In [2]:
# 合并所有解析出来的经纬度数据文件
def get_filenames(pattern):

    # 目标目录路径，根据实际情况替换，比如 "/soil_data/result/"
    target_dir = "./"
    # pattern = r'^[MRW].*(csv)$' 
    result = []
    # 遍历目录下的文件
    for filename in os.listdir(target_dir):
        if re.match(pattern, filename):
            result.append(filename)
            
    return(result)

In [3]:
tem_reference1 = "2018年平均温度(grids).csv"
tem_reference2 = "2019年平均温度(grids).csv"
df1 = pd.read_csv(tem_reference1)
df2 = pd.read_csv(tem_reference2)

In [4]:
pattern = r'^[MRW].*(csv)$'
filenames = get_filenames(pattern)

In [5]:
result_df = pd.DataFrame()
for filename in filenames:

    # 记录开始时间
    start_time = time.time()  # 返回当前时间的时间戳（秒）
            
    # 读取数据
    df3 = pd.read_csv(filename)
    
    average_tems = []
    for k in range(len(df1)):
        
        start = df3.loc[df3['grid_num'] == df1.loc[k, 'grid_num'], 'start_month'].iloc[0]
        end = df3.loc[df3['grid_num'] == df1.loc[k, 'grid_num'], 'end_month'].iloc[0]
    
        if np.isfinite(start) and np.isfinite(end):
            
            start = int(start)
            end = int(end)
    
            if start<=end:
                average_tems.append(round(sum(df1.values[k,start:end+1])/(end-start+1),2))
            else:
                average_tems.append(round((sum(df1.values[k,start:13])+sum(df2.values[k,1:end+1]))/(end-start+13),2)) #出现跨年情形
        else:
            average_tems.append(np.nan)
    
    file_df = pd.DataFrame({'grid_num':df1['grid_num'], f'average_tem_{filename.rsplit('.',1)[0]}':average_tems})
    # 合并数据：首次循环直接赋值，后续按经纬度关联合并
    if result_df.empty:
        result_df = file_df
    else:
        # 按经纬度匹配合并，确保同一经纬度的月份数据对应
        result_df = result_df.merge(file_df, on='grid_num', how='outer')

    # 记录结束时间
    end_time = time.time()
    # 计算耗时
    elapsed_time = end_time - start_time
    print(f"耗时: {elapsed_time/60:.2f} 分")
    
# result_df.to_csv('2018年各种农作物平均温度(汇总).csv', index=False)

耗时: 0.81 分
耗时: 0.25 分
耗时: 0.58 分
耗时: 0.64 分
耗时: 0.69 分
耗时: 0.26 分


In [6]:
# result_df[result_df[result_df.columns[1:]].values>0]

In [7]:
results = pd.DataFrame(columns=result_df.columns)
results['grid_num'] = np.array(list(range(1, 360*720+1)))

In [8]:
merged_df = result_df.set_index('grid_num').combine_first(results.set_index('grid_num')).reset_index()
merged_df

,grid_num,average_tem_Wheat.2(grids),average_tem_Rice.2(grids),average_tem_Wheat(grids),average_tem_Maize(grids),average_tem_Rice(grids),average_tem_Maize.2(grids)
0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
259195,259196,NaN,NaN,NaN,NaN,NaN,NaN
259196,259197,NaN,NaN,NaN,NaN,NaN,NaN
259197,259198,NaN,NaN,NaN,NaN,NaN,NaN
259198,259199,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
merged_df.to_csv('2018年各种农作物平均温度(汇总).csv', index=False)

In [10]:
merged_df[merged_df[merged_df.columns[1:]].values>0]

,grid_num,average_tem_Wheat.2(grids),average_tem_Rice.2(grids),average_tem_Wheat(grids),average_tem_Maize(grids),average_tem_Rice(grids),average_tem_Maize.2(grids)
49184,49185,4.57,NaN,NaN,6.37,7.10,NaN
49184,49185,4.57,NaN,NaN,6.37,7.10,NaN
49184,49185,4.57,NaN,NaN,6.37,7.10,NaN
49185,49186,4.42,NaN,7.18,6.27,6.98,NaN
49185,49186,4.42,NaN,7.18,6.27,6.98,NaN
...,...,...,...,...,...,...,...
245207,245208,-4.87,NaN,NaN,NaN,0.58,NaN
245208,245209,-5.29,NaN,NaN,NaN,0.18,NaN
245209,245210,-5.21,NaN,NaN,NaN,0.23,NaN
245210,245211,-5.40,NaN,NaN,NaN,0.08,NaN
